In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import preprocessing as pr

In [2]:
df = pd.read_csv('test.csv', sep=";")

In [3]:
df = df.drop(['USER_ID', 'PORTED_OUT', 'OBLIG_ON_START'], axis='columns')

# ACT_DATE

In [4]:
# ACT_DATE в количество дней ACT_DAYS
df['ACT_DAYS'] = (pd.to_datetime('2016-12-01') - pd.to_datetime(df['ACT_DATE'])).dt.days

In [5]:
# ACT_DATE в количество месяцев ACT_MONTHS
df['ACT_MONTHS'] = (pd.to_datetime('2016-12-01') - 
                    pd.to_datetime(df['ACT_DATE'])) / np.timedelta64(1, 'M')

In [6]:
df = df.drop(['ACT_DATE'], axis='columns')

# STATUS (2 variants)

In [7]:
# first variant(D and others)
df.loc[df.STATUS == 'D', 'STATUS'] = 1
df.loc[df.STATUS != 'D', 'STATUS'] = 0

In [8]:
# second variant(3 categories: 
# normal activity(2) - D
# low activity(1) - F, Q, R and W
# innactivity(0) - Z, L and E
for stat in df.STATUS.unique():
    if stat == 'D':
        rating = 2
    elif stat in ['F', 'Q', 'R', 'W']:
        rating = 1
    else:
        rating = 0
    df.loc[df.STATUS == stat, 'STATUS'] = rating

# TP_CURRENT

In [9]:
# replace by the average REVENUE for each category
mean_revenue = df.groupby('TP_CURRENT').mean()['REVENUE_NOV_16']
for tp in df.TP_CURRENT.unique():
    df.loc[df.TP_CURRENT == tp, 'TP_CURRENT'] = mean_revenue[tp]

# TP_CHANGES_NUM (2 variants)

In [10]:
# replace those who change tariff a lot for the average number of tariff changes
small_values = df[df.groupby('TP_CHANGES_NUM')
                 .TP_CHANGES_NUM.transform('count') <= 100].TP_CHANGES_NUM
small_value = np.round(small_values.mean())
df.loc[df.groupby('TP_CHANGES_NUM')
                 .TP_CHANGES_NUM.transform('count') <= 100, 
       'TP_CHANGES_NUM'] = small_value

df.TP_CHANGES_NUM.value_counts()

0.0    16828
1.0     2361
2.0      563
3.0      155
7.0       93
Name: TP_CHANGES_NUM, dtype: int64

In [11]:
# add column "TP_IS_CHANGED" which shows it is changed tp or no
df.loc[df.TP_CHANGES_NUM > 0, 'TP_IS_CHANGED'] = 1
df.loc[df.TP_CHANGES_NUM == 0, 'TP_IS_CHANGED'] = 0
df.TP_IS_CHANGED.value_counts()

0.0    16828
1.0     3172
Name: TP_IS_CHANGED, dtype: int64

# START_PACK and OFFER_GROUP

In [12]:
df = pd.concat([df, pd.get_dummies(df.START_PACK, prefix = 'SP')], axis = 1)
df = pd.concat([df, pd.get_dummies(df.OFFER_GROUP, prefix = 'OG')], axis = 1)
df = df.drop(['OFFER_GROUP', 'START_PACK'], axis='columns')

# BIRTHDAY(4 variants)

In [13]:
# new column with age in years
df['AGE'] = np.round((pd.to_datetime('2016-12-01') -
                      pd.to_datetime(df['BIRTHDAY']))/np.timedelta64(1, 'Y'))

In [14]:
# 1 variant(filna) - fill NaNs with the median for gender
df.loc[(df['GENDER'] == 'F') &  
       df['AGE'].isna(), 'AGE'] = df[df['GENDER'] == 'F']['AGE'].median()
df.loc[(df['GENDER'] == 'M') &  
       df['AGE'].isna(), 'AGE'] = df[df['GENDER'] == 'M']['AGE'].median()

In [15]:
# 2 variant(filna) -fFill NaNs with the median for TP_CURRENT
for tp in df.TP_CURRENT.unique():
    df.loc[(df['TP_CURRENT'] == tp) &  
           df['AGE'].isna(), 'AGE'] = df[df['TP_CURRENT'] == tp]['AGE'].median()

In [16]:
# 1 variant(outliers) - 3*std
# fill those that aren't in 3 standard deviations with median
df[df['AGE'] < df['AGE'].median() + 3*df['AGE'].std()].loc[:, 'AGE'] = df['AGE'].median()

In [17]:
# 2 variant(outliers) - 99%
# Fill those that aren't in quantile = 99% with median
perc = 0.99
df[df['AGE'] < df['AGE'].quantile(perc)].loc[:, 'AGE'] = df['AGE'].median()

In [18]:
df = df.drop(['BIRTHDAY'], axis='columns')


In [59]:
# three categories 

kbins = pr.KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
kbins.fit(np.array(df.AGE).reshape(-1, 1)) 
df.AGE_GROUPS = pd.Series(np.ndarray.flatten(
    kbins.transform(np.array(df.AGE).reshape(-1, 1))))

df.groupby(['AGE', 'AGE_GROUPS'])

KeyError: 'AGE_GROUPS'

# GENDER 

In [19]:
# fill NaNs in "GENDER"
df.loc[df.GENDER.isna() & 
       (df.OUTGOING_NOV_16 < df.OUTGOING_NOV_16.mean()), 'GENDER'] = 'M'
df.loc[df.GENDER.isna() & 
       (df.OUTGOING_NOV_16 >= df.OUTGOING_NOV_16.mean()),'GENDER'] = 'F'
df.loc[df.GENDER.isna(), 'GENDER'] = 'M'

df.loc[df.GENDER == 'F', 'GENDER'] = 0
df.loc[df.GENDER == 'M', 'GENDER'] = 1
df.GENDER.value_counts(dropna=False)

1    11135
0     8434
Name: GENDER, dtype: int64

# MLLS_STATE

In [20]:
# change 'MLLS_STATE' (add flag a member or not and change categories to numbers)
# del = -1
# null = 0
# begin = 1
# active = 2
mask = df.MLLS_STATE == 'Active'
df.loc[mask, 'IN_MLLS'] = 1
df.loc[mask, 'MLLS_STATE'] = 2

mask = df.MLLS_STATE == 'Begin'
df.loc[mask, 'IN_MLLS'] = 1
df.loc[mask, 'MLLS_STATE'] = 1

mask = df.MLLS_STATE.isna()
df.loc[mask, 'IN_MLLS'] = 0
df.loc[mask, 'MLLS_STATE'] = 0

mask = df.MLLS_STATE == 'Deleted'
df.loc[mask, 'IN_MLLS'] = 0
df.loc[mask, 'MLLS_STATE'] = -1

df.MLLS_STATE.value_counts()

 0    16426
 2     2903
 1      209
-1       31
Name: MLLS_STATE, dtype: int64

# PORTED_IN(2 variants)

In [21]:
# 1 variant
df = df.drop(['PORTED_IN'], axis='columns')

In [22]:
# 2 varinat
# nothing to do

# OBLIG_NUM(2 variants)

In [23]:
# 1 variant(0, 1, 2)
df.OBLIG_NUM = df.OBLIG_NUM.fillna(0)
df.OBLIG_NUM.value_counts(dropna=False)

0.0    16126
1.0     3394
2.0       49
Name: OBLIG_NUM, dtype: int64

In [24]:
# 2 variant - change values on two: 1 - bought/ 0 - no
df.OBLIG_NUM = df.OBLIG_NUM.fillna(0)
df.loc[df.OBLIG_NUM == 2, 'OBLIG_NUM'] = 1

df.OBLIG_NUM.value_counts()

0.0    16126
1.0     3443
Name: OBLIG_NUM, dtype: int64

# ASSET_TYPE_LAST and DEVICE_TYPE_BUS

In [25]:
# if ASSET_TYPE_LAST == DEVICE_TYPE_BUS then item==1 else 0
df.loc[df['ASSET_TYPE_LAST'] == df['DEVICE_TYPE_BUS'], 'ASSET_TYPE_LAST'] = 1
df.loc[df['ASSET_TYPE_LAST'] != 1, 'ASSET_TYPE_LAST'] = 0
df.ASSET_TYPE_LAST.value_counts(dropna=False)

0    16839
1     2730
Name: ASSET_TYPE_LAST, dtype: int64

In [26]:
#defise type on 4 categories
df.loc[df['DEVICE_TYPE_BUS'] == 'Tablet', 'DEVICE_TYPE_BUS'] = 'Other_device'
df.loc[df['DEVICE_TYPE_BUS'] == 'Modem/Router', 'DEVICE_TYPE_BUS'] = 'Other_device'
df = pd.concat([df, pd.get_dummies(df.DEVICE_TYPE_BUS)], axis = 1)
df = df.drop(['DEVICE_TYPE_BUS'], axis='columns')

# USAGE_AREA

In [27]:
df = pd.concat([df, pd.get_dummies(df.USAGE_AREA)], axis = 1)
df = df.drop(['USAGE_AREA'], axis='columns')

# REFILL and REVENUE

In [28]:
df.REFILL_NOV_16 = df.REFILL_NOV_16.fillna(0)
df.REFILL_OCT_16 = df.REFILL_OCT_16.fillna(0)
df.REVENUE_OCT_16 = df.REVENUE_OCT_16.fillna(0)
df.REVENUE_NOV_16 = df.REVENUE_NOV_16.fillna(0)

df.REFILL_OCT_16 = df.REFILL_OCT_16.div(10000)
df.REFILL_NOV_16 = df.REFILL_NOV_16.div(10000)


df.REFILL_NOV_16.value_counts()

0.0      8984
3.0      1087
5.0       981
10.0      924
6.0       623
         ... 
146.0       1
93.0        1
200.0       1
281.0       1
135.0       1
Name: REFILL_NOV_16, Length: 107, dtype: int64

# OUTGOING and GPRS

In [29]:
df.OUTGOING_NOV_16 = df.OUTGOING_NOV_16.fillna(0)
df.OUTGOING_OCT_16 = df.OUTGOING_OCT_16.fillna(0)
df.GPRS_OCT_16 = df.GPRS_OCT_16.fillna(0)
df.GPRS_NOV_16 = df.GPRS_NOV_16.fillna(0)

In [30]:
# add new categories:
# - use internet and use calls
# - use only internet
# - use only calls
# - don't use anything
mask_outg = (df.OUTGOING_NOV_16 > 0) & (df.OUTGOING_OCT_16 > 0)
mask_gprs = (df.GPRS_OCT_16 > 0) & (df.GPRS_NOV_16 > 0)
df.loc[mask_outg & mask_gprs, 'INTERNET_AND_CALLS'] = 'NET_AND_CALLS'
df.loc[~mask_outg & mask_gprs, 'INTERNET_AND_CALLS'] = 'ONLY_NET'
df.loc[mask_outg & ~mask_gprs, 'INTERNET_AND_CALLS'] = 'ONLY_CALLS'
df = pd.concat([df, pd.get_dummies(df.INTERNET_AND_CALLS)], axis = 1)
df = df.drop(['INTERNET_AND_CALLS'], axis='columns')

In [31]:
df.REVENUE_NOV_16.value_counts(dropna=False)

0.000000     7105
3.000000      579
5.000000      211
4.000000      121
6.500000      115
             ... 
90.890000       1
23.240000       1
10.719298       1
1.502930        1
0.202070        1
Name: REVENUE_NOV_16, Length: 3372, dtype: int64

In [32]:
df

,STATUS,TP_CURRENT,TP_CHANGES_NUM,GENDER,MLLS_STATE,OBLIG_NUM,ASSET_TYPE_LAST,REFILL_OCT_16,REFILL_NOV_16,OUTGOING_OCT_16,...,Undefined,Countryside,Local Towns,Minsk,Mix,Regional Cities,Undefined,NET_AND_CALLS,ONLY_CALLS,ONLY_NET
0,0,6.19448,0.0,1,0,0.0,0,0.0,0.0,0.000000,...,0,0,0,0,0,1,0,0,0,0
1,0,3.64031,0.0,0,0,0.0,0,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,0,0,0
2,0,3.32828,0.0,0,0,0.0,0,15.0,0.0,59.483333,...,0,1,0,0,0,0,0,1,0,0
3,0,12.5699,0.0,1,0,0.0,0,0.0,0.0,0.000000,...,1,0,0,0,0,0,0,0,0,0
4,0,6.14527,0.0,0,0,0.0,0,6.0,9.0,260.200000,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,6.5263,2.0,1,2,0.0,0,7.0,10.0,40.016667,...,0,0,0,0,1,0,0,1,0,0
19996,0,2.78235,0.0,0,0,0.0,0,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,0,0,0
19997,0,12.5699,0.0,1,0,0.0,0,10.0,10.0,0.000000,...,1,0,0,0,0,0,1,0,0,0
19998,0,4.43371,0.0,1,0,1.0,1,6.0,6.0,320.766667,...,0,0,0,0,0,1,0,1,0,0
